# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

import os
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
gkey = os.getenv("gkey")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
file = pd.read_csv('../output_data/city.csv')
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Ushuaia,75,AR,1630093926,52,-54.8000,-68.3000,42.42,16.11
1,1,Nosy Varika,43,MG,1630093926,85,-20.5833,48.5333,66.36,3.49
2,2,Broken Hill,2,AU,1630093927,52,-31.9500,141.4333,42.51,10.36
3,3,Torbay,90,CA,1630093927,91,47.6666,-52.7314,66.27,24.16
4,4,Chifeng,0,CN,1630093844,71,42.2683,118.9636,55.78,8.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
gmaps.configure(api_key = gkey)

In [44]:
locations = file[['Latitude', 'Longitude']]
humidity = file['Humidity'].astype(float)

In [45]:
fig = gmaps.figure(center = [31.9686, -99.9018], zoom_level = 2)

In [46]:
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [47]:
weather = file.loc[(file['Max Temp'] >= 58) & (file['Max Temp'] >= 77)]
wind = weather.loc[(weather['Wind Speed'] <= 7)]
ideal = wind.loc[(wind['Cloudiness'] <= 25)]
len(ideal)

40

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
hotel_df = ideal.reset_index(drop = True)
hotel_df['Hotel Name'] = ""

#param to update
parameters = {
    'radius': 1500,
    'types': 'hotels',
    'keyword':'hotels',
    'key': 'g_key'
}

In [49]:
for index, row in hotel_df.iterrows():
    Latitude = row['Latitude']
    Longitude = row['Longitude']
    parameters['location'] = f'{Latitude} , {Longitude}'
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    hotel_name = requests.get(base_url, params = parameters)
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent = 4, sort_keys = True))
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']
    except (KeyError, IndexError):
        print('....Missing Data.....Skippping.......')

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
....Missing Data.....Skippping.......
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
....Missing Data.....Skippping.......
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
....Missing Data.....Skippping.......
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
....Missing Data.....Skippping.......
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
....Missing Data.....Skippping.......
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
   

In [50]:
 hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,11,Bethel,1,US,1630093929,69,41.3712,-73.4140,92.80,3.29,
1,24,Lincoln,1,US,1630093717,45,40.8000,-96.6670,98.13,3.00,
2,28,Altamont,1,US,1630093935,33,42.2068,-121.7372,77.77,4.00,
3,37,Ponta do Sol,21,PT,1630093939,73,32.6667,-17.1000,77.00,4.74,
4,65,Westport,20,US,1630093948,63,41.1415,-73.3579,94.08,3.00,
5,83,Mayo,1,US,1630093954,75,38.8876,-76.5119,95.36,7.00,
6,96,George Town,20,MY,1630093850,91,5.4112,100.3354,78.89,1.01,
7,113,Half Moon Bay,20,US,1630093963,66,37.4636,-122.4286,100.06,4.61,
8,141,Caceres,2,ES,1630093973,28,39.5220,-6.3748,81.81,6.73,
9,154,Shingū,2,JP,1630093977,84,33.7333,135.9833,77.92,4.70,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
